In [16]:
import cv2 as cv
import re

import pathlib

In [24]:
path = pathlib.Path('./CERUG')
files = [fp for fp in path.iterdir()]

In [32]:
pathname = str(files[1])
parts = pathname.split('\\')
print("parts of filepath: ", parts)
filename = parts[-1]
print("filename: ", filename)

re.search(r'(\d*)_(\d*)(-\d*)?', filename)[0]

parts of filepath:  ['CERUG', 'Writer0202_03-02.ppm']
filename:  Writer0202_03-02.ppm


'0202_03-02'

In [35]:
list(map(lambda x: re.search(r'(\d*)_(\d*)(-\d*)?', str(x))[1], files))

['0202', '0202', '0909']

In [10]:
im = cv.imread(str(path.iterdir().__next__()))

In [ ]:
for fp in path.iterdir():
    fp.